In [31]:
import random
from milvus import Milvus, IndexType, MetricType, Status
# milvus Tests Env

In [32]:
# This program demos how to connect to Milvus vector database, 
# create a vector collection,
# insert 10 vectors, 
# and execute a vector similarity search.

# Milvus server IP address and port.
# You may need to change _HOST and _PORT accordingly.
_HOST = '127.0.0.1'
_PORT = '19530'  # default value
# _PORT = '19121'  # default http value
# Vector parameters
_DIM = 8  # dimension of vector
_INDEX_FILE_SIZE = 32  # max file size of stored index

In [40]:
def main():
    # Specify server addr when create milvus client instance
    # milvus client instance maintain a connection pool, param
    # `pool_size` specify the max connection num.
    milvus = Milvus(_HOST, _PORT)

    # Create collection demo_collection if it dosen't exist.
    collection_name = 'example_collection_'

    status, ok = milvus.has_collection(collection_name)
    if not ok:
        param = {
            'collection_name': collection_name,
            'dimension': _DIM,
            'index_file_size': _INDEX_FILE_SIZE,  # optional
            'metric_type': MetricType.L2  # optional   chosing our own Metrix Type
        }
        
        milvus.create_collection(param)

    # Show collections in Milvus server
    _, collections = milvus.list_collections()
    print("Show collections in Milvus server")
    print(collections)
    
    
    # Describe demo_collection
    _, collection = milvus.get_collection_info(collection_name)
    print("Describe demo_collection")
    print(collection)

    # 10000 vectors with 128 dimension
    # element per dimension is float32 type
    # vectors should be a 2-D array
    vectors = [[random.random() for _ in range(_DIM)] for _ in range(10)]
    print("Vector DIM is _DIM, and we create 10 vectors with dim 8" )
    for i in range(0,10):
        print(['%.2f'%(vectors[i][j]) for j in range(_DIM)])
    # You can also use numpy to generate random vectors:
    #   vectors = np.random.rand(10000, _DIM).astype(np.float32)

    # Insert vectors into demo_collection, return status and vectors id list
    status, ids = milvus.insert(collection_name=collection_name, records=vectors)
    if not status.OK():
        print("Insert failed: ".format(status))
    # Transfer our Data into milvus .
    
    
    # Flush collection  inserted data to disk.
    
    
    milvus.flush([collection_name])
    
    
    # Get demo_collection row count
    status, result = milvus.count_entities(collection_name)

    # present collection statistics info， 获取相应的统计信息
    _, info = milvus.get_collection_stats(collection_name)
    print(info)

    # Obtain raw vectors by providing vector ids
    status, result_vectors = milvus.get_entity_by_id(collection_name, ids[:10])

    
    # create index of vectors, search more rapidly
    index_param = {
        'nlist': 2048
    }

    # Create ivflat index in demo_collection
    # You can search vectors without creating index. however, Creating index help to
    # search faster
    print("Creating index: {}".format(index_param))
    status = milvus.create_index(collection_name, IndexType.IVF_FLAT, index_param)

    # describe index, get information of index
    status, index = milvus.get_index_info(collection_name)
    print(index)
    
    
    print("QUERY___________________________________________________")
    # Use the top 10 vectors for similarity search  使用相似性检索功能，定义是个QUERY
    query_vectors = vectors[1:5]
    for i in range(len(query_vectors)):
        print(["%.2f"%query_vectors[i][j] for j in range(_DIM)])
    
    # execute vector similarity search
    search_param = {
        "nprobe": 16
    }

    print("Searching ... ")

    param = {
        'collection_name': collection_name,
        'query_records': query_vectors,
        'top_k': 2,
        'params': search_param,
    }

    status, results = milvus.search(**param)
    if status.OK():
        # indicate search result
        # also use by:
        #   `results.distance_array[0][0] == 0.0 or results.id_array[0][0] == ids[0]`
        if results[0][0].distance == 0.0 or results[0][0].id == ids[0]:
            print('Query result is correct')
        else:
            print('Query result isn\'t correct')

        # print results
        print(results)
    else:
        print("Search failed. ", status)

    # Delete demo_collection
    status = milvus.drop_collection(collection_name)
    print(results)
    print(type(results[0][0].id))
    print(type(results[0][1].id))



In [43]:
%%time
main()

Show collections in Milvus server
['three_dimension_demo', 'MinHang_Law_Vector_Test', 'MinHang_Law_Vector_Test_728', 'MinHang_Law_Vector_Test_768', 'example_collection_']
Describe demo_collection
CollectionSchema(collection_name='example_collection_', dimension=8, index_file_size=32, metric_type=<MetricType: L2>)
Vector DIM is _DIM, and we create 10 vectors with dim 8
['0.78', '0.23', '0.23', '0.71', '0.37', '0.67', '0.13', '0.59']
['0.94', '0.71', '0.76', '0.95', '0.89', '0.08', '0.88', '0.21']
['0.51', '0.80', '0.43', '0.31', '0.71', '0.41', '0.01', '0.68']
['0.58', '0.90', '0.68', '0.96', '0.49', '0.16', '0.91', '0.98']
['0.07', '0.16', '0.17', '0.60', '0.81', '0.67', '0.88', '0.33']
['0.81', '0.64', '0.73', '0.36', '0.44', '0.75', '0.26', '0.90']
['0.47', '0.10', '0.34', '0.94', '0.44', '0.44', '0.20', '0.40']
['0.80', '0.33', '0.13', '0.19', '0.33', '0.88', '0.55', '0.55']
['0.73', '0.45', '0.95', '0.77', '0.83', '0.85', '0.83', '0.58']
['0.06', '0.64', '0.94', '0.97', '0.57', '0.

In [42]:
    # {0: [0.03410822597897978, 0.5158045625422135, 0.37750077183003106],
    #  1: [0.21156951459788342, 0.7371109592547848, 0.16279911514021572],
    #  2: [0.5988454884530987, 0.4958539058846362, 0.6789252800361267],
    #  3: [0.35504005558141316, 0.670745582582586, 0.44967594323739524],
    #  4: [0.32605080182810764, 0.6853550771457717, 0.4204260632314877],
    #  5: [0.03556487883739212, 0.6030845254408221, 0.2578816256632094],
    #  6: [0.17870902743702655, 0.8256214440497556, 0.6870864964626284],
    #  7: [0.7152150982456168, 0.40622147870825076, 0.5977015696432396],
    #  8: [0.05042273222352511, 0.8985288492583241, 0.4635093585523088],
    #  9: [0.6549209269469661, 0.5903931573833446, 0.9061315453421189]}
    
status, results = milvus.search(**param)

NameError: name 'milvus' is not defined

In [48]:
milvus = Milvus(_HOST, _PORT)
_, collections = milvus.list_collections()
print(collections)

#status, index = milvus.get_index_info(collection_name)
#print(index)

['three_dimension_demo', 'MinHang_Law_Vector_Test', 'MinHang_Law_Vector_Test_728', 'MinHang_Law_Vector_Test_768']
